### Prometheus Installation

Prometheus ist am einfachsten mittels [helm](https://helm.io) installierbar.


In [ ]:
%%bash
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm upgrade --install monitoring prometheus-community/kube-prometheus-stack -n monitoring --namespace monitoring --create-namespace

Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana) 
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

sind nur innerhalb des Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen Aussen öffnen:

In [ ]:
%%bash
kubectl -n monitoring patch service monitoring-grafana                      -p '{"spec": {"type": "NodePort"}}'
kubectl -n monitoring patch service monitoring-kube-prometheus-prometheus   -p '{"spec": {"type": "NodePort"}}'
kubectl -n monitoring patch service monitoring-kube-prometheus-alertmanager -p '{"spec": {"type": "NodePort"}}'

Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [ ]:
%%bash
echo "Grafana UI     : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-grafana -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "- User         : admin"
echo "- Password     : $(kubectl --namespace monitoring get secrets monitoring-grafana -o jsonpath="{.data.admin-password}" | base64 -d ; echo)"
echo "Prometheus UI  : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')
echo "Alertmanager UI: " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')

**Grafana UI** Auf der linken Seite kann zwischen einer Reihe von vorbereitenden Dashboards ausgewählt werden, z.B. Nodes.

In der **Prometheus** Oberfläche kann mittels der Abfragesprache [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/) gezielt Ressourcen ausgewählt werden, z.B. durch Query von `apiserver_storage_objects`.

Der **Alertmanager** dient zum Verarbeiten von Ausnahmen, z.B. 


In [ ]:
%%bash
kubectl apply -f - <<EOF 
apiVersion: monitoring.coreos.com/v1
kind: PrometheusRule
metadata:
  name: node-high-cpu-usage
  namespace: monitoring  # Passe das Namespace entsprechend deiner Umgebung an
  labels:
    prometheus: kube-prometheus
    role: alert-rules
spec:
  groups:
  - name: node-alerts
    interval: 30s
    rules:
    - alert: HighCPUUsage
      expr: 100 - (avg by(instance) (rate(node_cpu_seconds_total{mode="idle"}[5m])) * 100) > 90
      for: 5m
      labels:
        severity: critical
      annotations:
        summary: "Hohe CPU-Auslastung auf {{ $labels.instance }}"
        description: "Die CPU-Auslastung auf {{ $labels.instance }} liegt über 90 % für mehr als 5 Minuten."
EOF


**Erklärung:**

1. **`apiVersion: monitoring.coreos.com/v1`** → Wird vom **Prometheus Operator** genutzt.
2. **`kind: PrometheusRule`** → Definiert eine Prometheus-Alerting-Rule.
3. **`metadata.name` & `metadata.namespace`** → Setzt den Namen und Namespace der Regel.
4. **`labels`** → Definiert Labels für die Regel, z. B. für eine bestimmte Prometheus-Instanz.
5. **`spec.groups`** → Gruppen für Regeln:
   - **`name: node-alerts`** → Gruppe von Alerting-Regeln für Nodes.
   - **`interval: 30s`** → Die Regeln werden alle 30 Sekunden geprüft.
6. **`rules`** → Die eigentliche Regel:
   - **`alert: HighCPUUsage`** → Name des Alarms.
   - **`expr`** → PromQL-Expression zur Überprüfung der CPU-Last.
   - **`for: 5m`** → Löst erst aus, wenn die Bedingung für 5 Minuten gilt.
   - **`labels`** → Labels für den Alarm, z. B. `severity: critical`.
   - **`annotations`** → Enthält eine Nachricht für den Alert.

- - -
### Custom Ressourcen

Der Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) stellt eine Reihe von [benutzerdefinierten Ressourcen (CRDs)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) für das Monitoring bereit.

* **Prometheus**, für eine Prometheus Instanz.
* **Alertmanager**, für eine Alertmanager Instanz
* **ServiceMonitor**, für die Überwachung von Kubernetes-Diensten.
* **PodMonitor**, für die Überwachung von Pods.


In [ ]:
%%bash
kubectl api-resources | grep monitoring
echo "---------------------------------"
kubectl explain servicemonitors                   